In [30]:
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import *
import torch as T
import torch.nn as nn
import random
import os
project_path= os.path.dirname(os.path.abspath(os.curdir))
import sys
sys.path.insert(0, project_path+ '/Tools')
sys.path.insert(1, project_path+ '/Optimal Control Methods/Learning Methods/Model Free RL')
sys.path.insert(2, project_path+ '/Systems')
from EnforceTyping import enforce_method_typing
from ParticlesandFields import Field, ClassicalParticle, ParticleInField
from DDPG import DDPGAgent, DDPGAlgorithm
T.Tensor.ndim = property(lambda self: len(self.shape))

In [31]:
coulomb_constant = 8.9875e9  # N*m^2/C^2
@dataclass(kw_only=True)
class ElectrostaticField2D(Field):
  """
  A class used to represent a 2D Electrostatic Field

  Attributes
  ----------
  field_sources: dict
      a formatted string to print out what the animal says
  dimensionality: tuple
      a tuple of the dimensionality of the field  

  Methods
  -------
  dynamics(self, observation_position: np.ndarray, time: float) -> np.ndarray:
      Represents the value of the field at any given point(s) or time. 
  potential(self, observation_position: np.ndarray, time: float) -> float:
      Represents the potential due to the field at a given position and/or time  
  potential_difference(self, initial_position: np.ndarray, final_position: np.ndarray, time: float) -> float:
      Represents the potential difference between two positions at a given time in the vector field   
  gradient(self, observation_position: np.ndarray, time: float) -> float:
      Represents the gradient at a given position and/or time in the vector field 
  curl(self, observation_position: np.ndarray, time: float) -> float:
      Represents the curl at a given position and/or time in the vector field 
  divergence(self, observation_position: np.ndarray, time: float) -> float:
      Represents the divergence at a given position and/or time in the vector field
  """
  field_sources: dict
  dimensionality: tuple = (2,)

  def __call__(self, observation_position: np.ndarray) -> np.ndarray:
      return self.dynamics(observation_position)

  def __post_init__(self):
    assert len(self.field_sources["Particle"]) == len(self.field_sources["Position"]), "The length of particles and fields don't match"
    for field_source, _ in zip(self.field_sources["Particle"], self.field_sources["Position"]):
        assert isinstance(field_source, ClassicalParticle),  "The field source is not a particle" 

  @enforce_method_typing
  def dynamics(self, observation_position: np.ndarray) -> np.ndarray:
      """
      This function outputs the field strength due to field sources experienced at any given point(s) or time. 
      This determines the physics of the field (a 2D Electricstatic Field in this case)

      Args:
          observation_position (np.ndarray): The position.

      Returns:
          np.ndarray: The electric field strength vector at the given position.
      """
      electric_field_vector = np.zeros_like(observation_position)
      for field_source, source_position in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          position_vectors = np.broadcast_to(source_position, reversed(observation_position.shape)).T
          displacement_vectors = observation_position - position_vectors
          displacement_magnitude = np.linalg.norm(displacement_vectors, axis=0)
          electric_field_vector += (displacement_vectors * field_source.charge) / displacement_magnitude**3
      electric_field_vector = coulomb_constant * electric_field_vector
      return np.round(electric_field_vector, 3)  # N/C or V/m

  @enforce_method_typing
  def potential(self, observation_position: np.ndarray) -> float:
      """
      Calculate the potential (voltage) at a position in the field.

      Args:
          observation_position (np.ndarray): The position.

      Returns:
          np.ndarray: The electric potential at the given position.
      """
      electric_potential = 0.0
      for field_source, source_position in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          position_vectors = np.broadcast_to(source_position, reversed(observation_position.shape)).T
          displacement_vectors = observation_position - position_vectors
          displacement_magnitude = np.linalg.norm(displacement_vectors, axis=0)
          electric_potential += field_source.charge / displacement_magnitude
      electric_potential = coulomb_constant * electric_potential
      return np.round(electric_potential, 3)  # V

  @enforce_method_typing
  def potential_difference(self, initial_position: np.ndarray, final_position: np.ndarray) -> float:
    """
    Calculate the potential difference between the initial position and the final position in the field.

    Args:
        initial_position (np.ndarray): The starting position.
        final_position (np.ndarray): The ending position.
        resolution (int, optional): The number of intervals to divide the path into. Defaults to 5000.

    Returns:
        float: The work required to move from the initial position to the final position.
    """
    assert initial_position.shape == self.dimensionality, "initial_position has the wrong dimensions"
    assert final_position.shape == self.dimensionality, "final_position has the wrong dimensions"
    PorentialDifference= self.potential(initial_position)- self.potential(final_position)
    return PorentialDifference

  @enforce_method_typing
  def gradient(self, observation_position: np.ndarray, delta: float= 0.001)->np.ndarray:
    """
    This function returns the derivative of the field at a given point

    Args:
        observation_position (np.ndarray): The position.
        delta (float, optional): The step size. Defaults to 0.001.

    Returns: 
      np.ndarray: The gradient of the field at the given position.
    """
    gradient= np.zeros_like(observation_position)
    for i in range(len(observation_position)):
      di= np.zeros_like(observation_position)
      di[i, ] = di[i, ]+delta
      plusdi= observation_position+ di
      minusdi= observation_position- di
      gradient[i]= (self.dynamics(plusdi)- self.dynamics(minusdi))[i]/ (2* delta)
    return gradient

  @enforce_method_typing
  def plot_field(self, low_bound= -5, high_bound= 5, n_vectors= 50):
    """
    This function plots the 2D electric vector field

    Args:
    low_bound (float, optional): The lower bound of the plot. Defaults to -5.
    high_bound (float, optional): The upper bound of the plot. Defaults to 5.
    n_vectors (int, optional): The number of vectors to plot. Defaults to 50.

    """
    observation_position= np.meshgrid(np.linspace(low_bound, high_bound, n_vectors), 
                                    np.linspace(low_bound, high_bound, n_vectors))
    observation_position= np.stack(observation_position)
    xd, yd = self.dynamics(observation_position)
    xd = xd / np.sqrt(xd**2 + yd**2)
    yd = yd / np.sqrt(xd**2 + yd**2)
    color_aara = np.sqrt(xd**2+ yd**2)
    fig, ax = plt.subplots(1,1)
    cp = ax.quiver(observation_position[0],observation_position[1],xd,yd,color_aara)
    fig.colorbar(cp)
    plt.rcParams['figure.dpi'] = 150
    plt.show()

In [32]:
negative_charge= ClassicalParticle(mass=1.0, charge= -1e-5)
positive_charge= ClassicalParticle(mass=1.0, charge= 1e-5)
sources = {"Particle": [negative_charge],
           "Position": [np.array([0.0, 0.0])]} 
target= np.array([-1.0, 0.0])
test_electric_field= ElectrostaticField2D(field_sources=sources)
point_charge_in_electric_field= ParticleInField(field=test_electric_field, 
                                               particle=negative_charge, 
                                               target_position=target)

In [33]:
critic_save_path= os.path.join(project_path, 'Data', 'Temp', 'critic_data', 'test_agent')
actor_save_path= os.path.join(project_path, 'Data', 'Temp', 'actor_data', 'test_agent')
test_actor_layer= (10, 5)
test_critic_layer= (10, 5)
test_actor_activations= (nn.ReLU(), nn.ReLU())
test_critic_activations= (nn.ReLU(), nn.ReLU())
observation_size= 4
action_size= 2
actor_learning_rate= 0.01
critic_learning_rate= 0.01
soft_update_rate=0.01
control_interval=0.5
control_magnitude=10.0
max_size= 64
batch_size= 16
test_agent = DDPGAgent(environment=point_charge_in_electric_field,
                       actor_layers=test_actor_layer, 
                       critic_layers=test_critic_layer,
                       actor_activations= test_actor_activations,
                       critic_activations= test_critic_activations,
                       observation_size= observation_size,
                       action_size= action_size,
                       actor_learning_rate= actor_learning_rate,
                       critic_learning_rate= critic_learning_rate,
                       soft_update_rate= soft_update_rate,
                       control_interval= control_interval,
                       control_magnitude= control_magnitude,
                       max_size= max_size,
                       batch_size= batch_size,
                       actor_save_path= actor_save_path,
                       critic_save_path= critic_save_path)

In [34]:
test_state= test_agent.observe() 
test_action= test_agent.act(test_state)
test_control= test_agent.control_mechanism(test_action)
test_value= test_agent.critic.forward(test_state, test_action)
# print(test_agent.policy)
print(test_state)
print(test_action)
print(test_control)
print(test_value)

tensor([3.5153, 6.6709, 0.0000, 0.0000])
tensor([-0.1242, -0.5197])
[-1.2989643 -5.1051865]
tensor([0.3276], grad_fn=<ViewBackward0>)


In [35]:
print(test_agent.memory)
curr_state= ParticleInField.State(position=np.array([1.0, 0.0]), 
                                  velocity=np.array([0.0, 0.0]))
for _ in range(64):
    rand_observation= test_agent.observe(curr_state)
    rand_action= test_agent.act(rand_observation)
    next_state, rand_reward, rand_terminal_signal= point_charge_in_electric_field.transition_step(curr_state, np.array(rand_action), test_agent.control_interval) 
    test_agent.memory.append((rand_observation, 
                              rand_action, 
                              test_agent.observe(next_state), 
                              rand_reward, 
                              rand_terminal_signal))
    curr_state= next_state
print(test_agent.memory)
(test_electric_field.dynamics(np.array([1.0, 0.0]))* positive_charge.charge)

deque([], maxlen=64)
deque([(tensor([1., 0., 0., 0.]), tensor([-0.3548, -0.0485]), tensor([ 1.0656, -0.0062,  0.2532, -0.0251]), -0.0655898122683154, False), (tensor([ 1.0656, -0.0062,  0.2532, -0.0251]), tensor([-0.2999, -0.0560]), tensor([ 1.2450, -0.0266,  0.4467, -0.0574]), -0.1795765746259934, False), (tensor([ 1.2450, -0.0266,  0.4467, -0.0574]), tensor([-0.1965, -0.1001]), tensor([ 1.5079, -0.0696,  0.5895, -0.1152]), -0.26371327924384147, False), (tensor([ 1.5079, -0.0696,  0.5895, -0.1152]), tensor([-0.1228, -0.1250]), tensor([ 1.8306, -0.1453,  0.6907, -0.1876]), -0.32547545283132795, False), (tensor([ 1.8306, -0.1453,  0.6907, -0.1876]), tensor([-0.0991, -0.1404]), tensor([ 2.1930, -0.2593,  0.7518, -0.2686]), -0.3691325641850898, False), (tensor([ 2.1930, -0.2593,  0.7518, -0.2686]), tensor([-0.0951, -0.1478]), tensor([ 2.5774, -0.4148,  0.7816, -0.3532]), -0.3978920777198809, False), (tensor([ 2.5774, -0.4148,  0.7816, -0.3532]), tensor([-0.0974, -0.1546]), tensor([ 2.9707

array([-0.89875,  0.     ])

In [36]:
batch = random.sample(test_agent.memory, test_agent.batch_size)
observations, actions, next_observations, rewards, dones = zip(*batch)

state = T.stack(observations).to(test_agent.critic.device)
print('state:', state)

action = T.stack(actions).to(test_agent.critic.device)
print('action:', action)

reward = T.tensor(rewards, dtype=T.float).unsqueeze(1).to(test_agent.critic.device)
print('reward:', reward)

new_state = T.stack(next_observations).to(test_agent.critic.device)
print('new_state:', new_state)

done = T.tensor(dones, dtype=T.float).unsqueeze(1).to(test_agent.critic.device)
print('done:', done)

target_actions = test_agent.actor.forward(new_state)
print('target_actions:', target_actions)

critic_value_ = test_agent.target_critic.forward(new_state, target_actions) 
print('critic_value:', critic_value_)

q_expected = test_agent.critic.forward(state, action)
print('q_expected:', q_expected)

q_targets = reward + test_agent.discount_rate * critic_value_ * (1 - done)
print('q_targets:', q_targets)

critic_loss = nn.MSELoss()(q_expected, q_targets.detach())
print('critic_loss:', critic_loss)

test_agent.critic.train()
test_agent.critic.optimizer.zero_grad()
critic_loss.backward()
test_agent.critic.optimizer.step()

mu = test_agent.actor.forward(state)
print('mu:', mu)

actor_loss = -test_agent.critic.forward(state, mu)
print('actor_loss:', actor_loss)

actor_loss = T.mean(actor_loss)
print('actor_loss mean:', actor_loss)

test_agent.actor.train()
test_agent.actor.optimizer.zero_grad()
actor_loss.backward()
test_agent.actor.optimizer.step()

test_agent.update_network_parameters()

state: tensor([[  6.6147,  -9.7569,  -0.1034,  -1.6468],
        [  2.9707,  -0.6133,   0.7888,  -0.4407],
        [ -9.5508, -41.7680,  -3.0314,  -3.9056],
        [  1.5079,  -0.0696,   0.5895,  -0.1152],
        [  0.8123, -26.4000,  -1.6099,  -2.9028],
        [  6.2856, -12.3826,  -0.3380,  -1.8553],
        [ -4.1527, -34.3813,  -2.3733,  -3.4768],
        [  1.5827, -24.9762,  -1.4714,  -2.7922],
        [  2.9248, -22.2877,  -1.2162,  -2.5874],
        [  6.4225,  -6.0597,   0.2494,  -1.3133],
        [-14.4865, -47.8561,  -3.5530,  -4.2093],
        [  5.4119,  -3.1805,   0.5491,  -0.9916],
        [  5.6728,  -3.6922,   0.4943,  -1.0552],
        [  1.8306,  -0.1453,   0.6907,  -0.1876],
        [-31.3617, -65.5121,  -4.8310,  -4.4419],
        [  2.5774,  -0.4148,   0.7816,  -0.3532]])
action: tensor([[-0.1559, -0.1322],
        [-0.1012, -0.1588],
        [-0.3424, -0.2056],
        [-0.1228, -0.1250],
        [-0.2929, -0.2321],
        [-0.1663, -0.1389],
        [-0.3211

In [ ]:
test_trajectory, test_trajectory_return= test_agent.sample_trajectory(10.0)
test_agent.plot_trajectory(test_trajectory)

In [ ]:
DDPGAlgorithm(point_charge_in_electric_field, test_agent, 800, 15)

In [ ]:
path, path_return= test_agent.sample_trajectory(10.0)
test_agent.plot_trajectory(path)